# About this notebook
This notebook will guide you through the analysis.

It will help you visualsie and setting the filtering parameters right, which you then can use for batch processing.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

from iob_ia.utils import filter_labels, visualise
from iob_ia.utils.classify import estimate_threshold
from iob_ia.utils.io_utils import (
    gen_out_path,
    read_image,
    read_labels,
    save_labels,
)
from iob_ia.utils.measure import overlapping_labels
from iob_ia.utils.segment import segment_3d_cellpose

## 1. Open an image
Define the path to an image

In [3]:
# For windows users: use "/" instead of "\" for file paths!
image_path = "G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/20250520_10x_Nq146um_405_488_640_01.vsi"

Next, we create an image object (this will not load the image data yet).

This image object has different attributes, e.g.:
- shape (size of the dimensions)
- metdata attributes, e.g. channel names or pixel/voxel size
Which we can directly access (and print out for our convinience)

(* for this image somehow the voxel size is incomplete, notice the "Z=None" below, which means the z-step of the stack is missing in the metadata)

In [4]:
image = read_image(image_path)
print("image shape (TCZYX):", image.shape)
print("channel names:", image.channel_names)
print("voxel size:", image.physical_pixel_sizes)

image shape (TCZYX): (1, 3, 373, 2048, 2048)
channel names: ['405 C', '488 C', '640 C']
voxel size: PhysicalPixelSizes(Z=None, Y=0.6500000000000001, X=0.6500000000000001)


---
## 2. Segmenting the image in 3D
Using cellpose we will create a segmenation mask.
(If you already have a segmentation mask you can also load it directly from file...)

In [5]:
# Starting with loading the image into memory (this may take a while, depending on the image size)
print('Loading image...')
img = image.get_data()
print(img.shape)

Loading image...
(3, 373, 2048, 2048)


The `segment_3d_cellpose` function will return a mask image, and works on single channel only.

It has several optional paramters that you can specify:

`segment_3d_cellpose(img, model_path, anisotropy, flow3D_smooth, cellprob_threshold)`
- `img`: the 3D single channel image array, e.g. with shape = (373, 2048, 2048)
- `model_path`: default is 'cpsam', but you could use your own cellpose model, e.g. 'C:/path/to/model'
- `anisotropy`: may help with anisotropic data (where ZYX voxel size is not uniform in all directions), e.g. voxel size = (1, 0.5, 0.5) > anisotropy = 2.0 (I did not observe great improvement with the tested images)
- `flow3D_smooth`: default = 0. Bigger values will smooth the cellpose flows with a gaussian (may help fusing smaller objects together)
- `cellprob_threshold`: cellpose parameter (default = 0.0). Values of -6 to +6; smaller values for more and larger objects.

In [7]:
# Segment the second channel (0-based indexing) of the image (that has 3 channels), using the default paramters, which work fine
mask = segment_3d_cellpose(img[1])

2025-09-12 14:41:26,196 [INFO] ** TORCH CUDA version installed and working. **
creating new log file
2025-09-12 14:41:26,198 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-09-12 14:41:26,199 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.13 
torch version:  	2.8.0+cu126
>>> GPU activated: True
2025-09-12 14:41:26,200 [WARNING] model_type argument is not used in v4.0.1+. Ignoring this argument...
2025-09-12 14:41:26,204 [INFO] ** TORCH CUDA version installed and working. **
2025-09-12 14:41:26,204 [INFO] >>>> using GPU (CUDA)
2025-09-12 14:41:28,951 [INFO] >>>> loading model C:\Users\Microscopy\.cellpose\models\cpsam
2025-09-12 14:41:55,536 [INFO] running YX: 373 planes of size (2048, 2048)
2025-09-12 14:57:19,235 [INFO] 100%|##########| 373/373 [15:23<00:00,  2.48s/it]
2025-09-12 14:57:26,748 [INFO] running ZY: 2048 planes of size (373, 2048)
2025-09-12 15:14:42,830 [INFO] 100%|##########| 2048/2048 [17:16<00:00,  1.98it/s]
2025

In [9]:
# Let's save this mask!
# first we define where to save the mask image
mask_path = gen_out_path(image_path, "mask")
print(mask_path)
# now we same the image
save_labels(mask, mask_path)
# short version would be
save_labels(mask, gen_out_path(image_path, "mask"))

G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_mask.tif
Saved labels to: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_mask.tif
Saved labels to: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_mask.tif


In [6]:
# If you have the mask already saved to file, load it directly
mask_path = gen_out_path(image_path, "mask") # or any other path e.g. "C:/path/to/mask.tif"
mask = read_labels(mask_path)

---
## 3. Visualising and finding your filtering options
cellpose does a good job in segmenting cells in 3D. But it is not perfect.

Let's visualise the images to napari. I have a plugin which will help you find some filtering options.

### 3.1 Visualise and empirically find filtering options

In [8]:
# load the origianl multichannel image (remember `img` is the image with all channels)
# we will load it with calibrated voxel units, since our image metadata is incomplete, we have to define it properly
voxel_size = (1.46, image.physical_pixel_sizes.Y, image.physical_pixel_sizes.X) # Z, Y, X
visualise.add_multichannel_image(img, name="raw_image", channel_names=image.channel_names, scale=voxel_size)

# let's add also the cellpose mask
visualise.add_labels(mask, name="cp_mask", scale=voxel_size)

In the napari viewer that opened, you can see the image and also toggle to 3D view if you like.

Under `Plugins` you can find the `Filter labels by properties` plugin. With this plugin you can interactively filter the cellpose objects by some properties (e.g. area (which is the volume in um3), or channel intensities).

Take note of what property you want to filter on and the values.

We segmented on the second channel (488), so we choose to filter also on intensities of the 488 channel. We found good filtering options for:
- area / volume values = 430 - 21'600 um3
- 488 mean intensity values = 200 - infinity

**Absolute intensity values** are generally a bad idea for (later) batch analysis, since it does not account for sample variability...

### 3.2 Measuring properties
Let's try to find a way to compensate for the sample variability by checking the property 'population'.

In [9]:
# First we measure the properties (verbose=True will report mean and median property values)
properties = filter_labels.measure_label_props(mask, img[1], voxel_size=voxel_size, verbose=True)

Measured properties of 9755 labels.
area: mean = 1525.9391215838039; median = 680.3855500000003
euler_number: mean = 0.07134802665299846; median = 1.0
extent: mean = 0.27616702097432266; median = 0.2916666666666667
intensity_mean: mean = 346.18783586807757; median = 267.0796703296703
intensity_min: mean = 146.718913377755; median = 124.0
intensity_max: mean = 696.7392106611994; median = 436.0
projected_area: mean = 72.50247257816508; median = 62.95250000000002
projected_convex_area: mean = 84.05350563813431; median = 74.36000000000003
projected_perimeter: mean = 33.17001349884242; median = 33.44629867976521
projected_circularity: mean = 0.6319078887264852; median = 0.6849776068149775


In [10]:
# We can get the mean or median values for a certain property as follows:
mean_intensity_mean = filter_labels.get_prop_mean(prop="intensity_mean", table=properties)
median_intensity_mean = filter_labels.get_prop_median(prop="intensity_mean", table=properties)
print("Mean intensity_mean =", mean_intensity_mean)
print("Median intensity_mean =", median_intensity_mean)

Mean intensity_mean = 346.18783586807757
Median intensity_mean = 267.0796703296703


Instead of using mean or median, we can also try to find a threshold that splits bimodal value distributions. 

E.g. assuming that we have 2 populations for the 488 channel mean intensity values (dark and bright objects)

In [11]:
mean_intensity_threshold = estimate_threshold(prop="intensity_mean", table=properties)
volume_threshold = estimate_threshold(prop="area", table=properties)
print("Threshold for 488 object mean intensity =", mean_intensity_threshold)
print("Threshold for object volume =", volume_threshold)

Threshold for 488 object mean intensity = 485.6809180149304
Threshold for object volume = 2476.13805022675


### 3.3. Creating a filtered mask

Let's filter objects using multiple property filters. For that, we can define a python dictionary with the properties and values we want to filter on.

Such dictionary should have keys = properties (as string), and min/max value ranges as array (i.e. `[min-value, max-value]`)

If you do not want to specify a min or max value you can either use `None` or `float("-inf")`/`float("inf")`. The latter specifying -/+ infinity.

If you use for both min max values = `None`, the min value will be set to the value returned by `estimate_threshold`, while the max value will be set to `float("inf")`.

In [12]:
# A filter dictionary with hard-coded volume size, and intensity_mean filter with dynamic min value (estimate_threshold value divided by 2) to infinity.
filter_dict = {
    "area": [430, 21600],
    "intensity_mean": [estimate_threshold("intensity_mean", properties) / 2, float("inf")],
}
print(filter_dict)

{'area': [430, 21600], 'intensity_mean': [242.8404590074652, inf]}


In [13]:
# Let's create a filtered mask using the defined filter properties in the dictionary above. (verbose=True would inform you on the calculated thresholds, in case you put [None, None] into the dictionary)
filtered_mask, updated_properties = filter_labels.filter_labels_by_property(mask, props_table=properties, props=filter_dict, verbose=True)

print("------------------------")
print("Before filtering we had", filter_labels.count_unique_labels(mask), "objects.")
print("After filtering we have", filter_labels.count_unique_labels(filtered_mask), "objects.")

Filtering on area between 430 and 21600
Removed 4352 labels. Remaining labels: 5403 (previously 9755)
Filtering on intensity_mean between 242.8404590074652 and inf
Removed 4397 labels. Remaining labels: 5358 (previously 9755)
Creating filtered mask...
------------------------
Before filtering we had 9755 objects.
After filtering we have 4029 objects.


**Q:** Why do the number differ... ("Removed x labels" and final object count):

**A:** We filter on the property table sequentially. The same object may be filtered/removed multiple times by the different filters.

---
## 4. Segmenting a second channel and checking for double positivity and total cell counts
You redo follow steps 2 and 3 for any other channel.
Here, I will redo these steps very breifly.
### 4.1 Segmenting and fitlering objects

In [14]:
# Define the file-path of the new mask before hand
mask_640_path = gen_out_path(image_path, "mask_" + image.channel_names[2])
# Segment the 3rd channel (0-based indexing), if the mask was not created yet
if os.path.exists(mask_640_path):
    print("The mask already exists => loading the mask from file")
    mask_640 = read_labels(mask_640_path)
else:
    mask_640 = segment_3d_cellpose(img[2])
    save_labels(mask_640, mask_640_path)

The mask already exists => loading the mask from file


In [15]:
# Measuring properties of the 640 channel objects
properties_640 = filter_labels.measure_label_props(mask_640, img[2], voxel_size=voxel_size, verbose=True)
mean_intensity_threshold_640 = estimate_threshold(prop="intensity_mean", table=properties_640)
print(">'Threshold' for 488 object mean intensity =", mean_intensity_threshold_640)
# To visualise in napari, you can un-comment the next line...
#visualise.add_labels(mask_640, name="640_cp_mask", scale=voxel_size)

Measured properties of 34286 labels.
area: mean = 495.2919766347783; median = 106.71505000000003
euler_number: mean = 0.20786910109082424; median = 1.0
extent: mean = 0.1497242138225513; median = 0.09353683333798635
intensity_mean: mean = 260.5319292277648; median = 145.324200913242
intensity_min: mean = 119.38887592603395; median = 73.0
intensity_max: mean = 462.8901300822493; median = 239.0
projected_area: mean = 40.85075679577671; median = 22.39250000000001
projected_convex_area: mean = 60.80420222539814; median = 37.180000000000014
projected_perimeter: mean = 28.24701562586959; median = 23.088529932111353
projected_circularity: mean = 0.4411056795495286; median = 0.37873824245202964
>'Threshold' for 488 object mean intensity = 21487.06852096016


Filterin the 640 channel objects:
- We can keep the same size restrictions as for the other channels (cell sizes usually don't change that much...)
- Filtering on intensity is a bit more tricky for this channel. I manually determined mean intensity in the range of 150-15000.

Below, I filter with those settings. (Could have taken half the of the channel intensity_mean...)

In [16]:
filter_dict_640 = {
    "area": [430, 21600],
    "intensity_mean": [150, float("inf")],
    #"intensity_mean": [filter_labels.get_prop_mean(prop="intensity_mean", table=properties_640) / 2, float("inf")], # for filtering min = intensity_mean / 2
    "intensity_max": [None, 15000],
}
# Filter the objects
filtered_mask_640, updated_propertie_640s = filter_labels.filter_labels_by_property(mask_640, props_table=properties_640, props=filter_dict_640, verbose=True)

print("------------------------")
print("Before filtering we had", filter_labels.count_unique_labels(mask_640), "objects.")
print("After filtering we have", filter_labels.count_unique_labels(filtered_mask_640), "objects.")

Filtering on area between 430 and 21600
Removed 26131 labels. Remaining labels: 8155 (previously 34286)
Filtering on intensity_mean between 150 and inf
Removed 18117 labels. Remaining labels: 16169 (previously 34286)
Filtering on intensity_max between -inf and 15000
Removed 97 labels. Remaining labels: 34189 (previously 34286)
Creating filtered mask...
------------------------
Before filtering we had 34286 objects.
After filtering we have 5972 objects.


---
### 4.2 Counting double positive objects
The approach I used here is as follows: if objects in channel 488 overlap at least by 50% with objects in channel 640, they are considered as a `640 positive 488 object(s)`.

We can create masks accordingly. The fuction below will do that and takes:
- first mask = objects of interest
- second mask = selector mask (to check if the first mask objects have overlap with the second mask)
- `min_overlap`: if not included here as a parameter, it defaults to 0.5 (=50%). But can be changed...

In [17]:
# 488 objects positive for 640
doublePos_mask488 = overlapping_labels(filtered_mask, filtered_mask_640, min_overlap=0.5)
# 640 objects positive for 488
doublePos_mask640 = overlapping_labels(filtered_mask_640, filtered_mask)

Summarise results:

In [18]:
count488_cp = filter_labels.count_unique_labels(mask)
count640_cp = filter_labels.count_unique_labels(mask_640)
count488 = filter_labels.count_unique_labels(filtered_mask)
count640 = filter_labels.count_unique_labels(filtered_mask_640)
count488_640 = filter_labels.count_unique_labels(doublePos_mask488)
count640_488 = filter_labels.count_unique_labels(doublePos_mask640)
# For convinience I put these results into a python dictionary, then convert it to a table
result = {
    "Image name": [image_path],
    "cellpose count-"+image.channel_names[1]: [count488_cp],
    "object count-"+image.channel_names[1]: [count488],
    "cellpose count-"+image.channel_names[2]: [count640_cp],
    "object count-"+image.channel_names[2]: [count640],
    image.channel_names[1]+"doublePos": [count488_640],
    image.channel_names[2]+"640doublePos": [count640_488]
}
result_table = pd.DataFrame.from_dict(result)
result_table

,Image name,cellpose count-488 C,object count-488 C,cellpose count-640 C,object count-640 C,488 CdoublePos,640 C640doublePos
0,G:/20241120_IOB_Magdalena/20250520_TestClearin...,9755,4029,34286,5972,1520,2234


Save this result table to file:

In [19]:
pd.DataFrame.to_csv(result_table, image_path + '.csv')

---
## 5. Visualise the results

In [20]:
# In case napari was closed we can re-open it with the original image...
visualise.add_multichannel_image(img, name="raw_image", channel_names=image.channel_names, scale=voxel_size)

# Now we add the created masks
visualise.add_labels(filtered_mask, name=image.channel_names[1]+"_mask", scale=voxel_size)
visualise.add_labels(filtered_mask_640, name=image.channel_names[2]+"_mask", scale=voxel_size)
visualise.add_labels(doublePos_mask488, name=image.channel_names[2]+"-pos-"+image.channel_names[1]+" objects", scale=voxel_size)
visualise.add_labels(doublePos_mask640, name=image.channel_names[1]+"-pos-"+image.channel_names[2]+" objects", scale=voxel_size)